In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#import numpy
#print(numpy.__version__)

In [24]:
#import sklearn
#print(sklearn.__version__)

1.2.2


In [ ]:
#import sys
#!{sys.executable} -m pip install -U --upgrade numpy

In [25]:
import sys
!{sys.executable} -m pip install -U --upgrade scikit-learn

Requirement already up-to-date: scikit-learn in /home/idies/miniconda3/envs/py38/lib/python3.8/site-packages (1.2.2)


In [ ]:
#from importlib import reload
#sklearn= reload(sklearn)

In [6]:
import sys
!{sys.executable} -m pip install scikit-learn-intelex

In [7]:
from sklearnex import patch_sklearn
patch_sklearn(global_patch=True)

Scikit-learn was successfully globally patched by Intel(R) Extension for Scikit-learn


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [8]:
import pandas as pd
import numpy as np
import os

In [9]:
df_pd = pd.read_csv('https://gitlab.com/mirsakhawathossain/exodata/-/raw/main/dataset/exodata.csv').sort_index(axis=0)

In [10]:
columns_1 = df_pd.columns[df_pd.mean()==1]
columns_2 = df_pd.columns[df_pd.mean()==0]
df_pd = df_pd.drop(columns=columns_1)
df_pd = df_pd.drop(columns=columns_2)

In [11]:
X = df_pd.drop(['exoplanet'],axis=1)

In [12]:
y = df_pd[['exoplanet']]

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, shuffle=True,stratify=y)

In [15]:
print(X_train.shape)
print(X_test.shape)

(45274, 741)
(30184, 741)


In [16]:
print(y_test.shape)
print(y_test.shape)

(30184, 1)
(30184, 1)


In [17]:
from sklearn.preprocessing import QuantileTransformer
qt = QuantileTransformer()

In [18]:
X_train = qt.fit_transform(X_train)
X_test = qt.transform(X_test)

In [19]:
from sklearn.model_selection import RepeatedKFold
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=41)

In [ ]:
#from importlib import reload
#sklearn= reload(sklearn)

In [28]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [21]:
clf = RandomForestClassifier(random_state=43,n_jobs = -1)

In [22]:
params = {'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
          'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10],
          'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
          'max_features': ['sqrt', 'log2', None],
          'criterion': ['gini', 'entropy'],
          'n_estimators': [50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200]}

In [31]:
hyp_se = HalvingGridSearchCV(estimator=clf,param_grid=params,random_state=45,scoring='accuracy',n_jobs=-1,cv=cv, verbose=3)

In [ ]:
hyp_se.fit(X_train, y_train)

n_iterations: 5
n_required_iterations: 11
n_possible_iterations: 5
min_resources_: 200
max_resources_: 45274
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 86400
n_resources: 200
Fitting 50 folds for each of 86400 candidates, totalling 4320000 fits


In [ ]:
# summarize result
print('Best Score: %s' %hyp_se.best_score_)
print('Best Hyperparameters: %s' %hyp_se.best_params_)
print('Best Estimator: %s' %hyp_se.best_estimator_)

In [ ]:
print("Test set accuracy score:", hyp_se.score(X_test, y_test))

dict